In [1]:
import os 
import pandas as pd 
import numpy as np 
import cv2

import torch 
import torch.nn as nn
import torch.optim as optim 
import torchvision
import albumentations as A

from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%pip install -q segmentation-models-pytorch

Note: you may need to restart the kernel to use updated packages.


# Config

In [3]:
batch_size = 4
lr = 1e-4
num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
best_val_loss = 9999999
display_step = 50

# Dataset

In [4]:
class NeolypsDataset(Dataset):
    def __init__(self, img_list, mask_list, transform = None):
        self.img_list = img_list
        self.mask_list = mask_list
        self.transform = transform
        
        print(f'Len of images: {len(self.img_list)}')
        print(f'Len of masks: {len(self.mask_list)}')
    
    def __len__(self):
        return len(self.img_list)
    
    def _read_mask(self, mask_path): 
        mask = cv2.imread(mask_path)
        
        # Convert to HSV
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)
        
        # Follow StackOverFlow: Red (0->10 and 160->179)
        lower1 = np.array([0, 100, 20])
        upper1 = np.array([10, 255, 255])
        
        lower2 = np.array([160, 100, 20])
        upper2 = np.array([179, 255, 255])
        
        # Follow StackOverFlow: Green (36 -> 70)
        lower_g = np.array([35,25,25])
        upper_g = np.array([70,255,255])
        
        red_mask = cv2.inRange(mask, lower1, upper1) + cv2.inRange(mask, lower2, upper2)
        green_mask = cv2.inRange(mask, lower_g, upper_g)
        
        red_mask[red_mask != 0] = 1 # Neopolyp pixel = 1
        green_mask[green_mask != 0] = 2 # Non neo polyp pixel = 2, background is 0
        
        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = full_mask.astype(np.int8)
        
        return full_mask
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]
        mask_path = self.mask_list[idx]
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        mask = self._read_mask(mask_path)
        
        if self.transform:
            transformer = self.transform(image=img, mask=mask)
            img = transformer['image'].float()
            mask = transformer['mask'].float()
        
        return img, mask

Create the dataset

In [5]:
img_dir = '/kaggle/input/bkai-igh-neopolyp/train/train'
mask_dir = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'

img_list = [
    os.path.join(img_dir, x) for x in os.listdir(img_dir)
]
mask_list = [
    os.path.join(mask_dir, x) for x in os.listdir(mask_dir)
]

Create the transform

In [6]:
train_transform = A.Compose(
    [
        A.Resize(256, 256, interpolation=cv2.INTER_LINEAR),
        A.VerticalFlip(p=0.3),
        A.HorizontalFlip(p=0.3),
        A.RGBShift(p=0.3),
        A.Normalize(),
        ToTensorV2()
    ]
)

val_transform = A.Compose(
    [
        A.Resize(256, 256, interpolation=cv2.INTER_LINEAR),
        A.Normalize(),
        ToTensorV2()
    ]
)

Train test split with ratio 0.8

In [7]:
train_size = int(0.9 * len(img_list))

train_dataset = NeolypsDataset(img_list[:train_size], mask_list[:train_size], transform=train_transform)
val_dataset = NeolypsDataset(img_list[train_size:], mask_list[train_size:], transform=val_transform)

Len of images: 900
Len of masks: 900
Len of images: 100
Len of masks: 100


Create the dataloader

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Training Phase

Model

In [9]:
import segmentation_models_pytorch as smp

model = smp.UnetPlusPlus(
    encoder_name="resnet50",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 99.9MB/s]


In [10]:
import wandb

wandb.login(key="8b9a21f3b6f5fbf21131e5a7c7d5bf22a2ad5f0c")

wandb.init(project="DL-Assignment", name=f'df-{num_epochs}-{batch_size}-{lr}')

# Training loop
train_loss = 0
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    
    for i, (img, label) in enumerate(train_dataloader):
        img = img.to(device)
        label = label.to(device)
        
        # Forward 
        label = label.squeeze(dim=1).long()
        outputs = model(img)
        
        # Loss 
        loss = criterion(outputs, label)
        
        # Backward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_loss += loss.item()
        
        if (i + 1) % display_step == 0:
            avg_train_loss = train_loss / display_step
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Train Loss: {avg_train_loss:.10f}")
            wandb.log({"Train loss": avg_train_loss})
            train_loss = 0
    
    model.eval()
    
    with torch.no_grad():
        val_loss = 0
        
        for img, label in val_dataloader:
            img = img.to(device)
            label = label.to(device)
            
            label = label.squeeze(dim=1).long()
            outputs = model(img)
            
            val_loss += criterion(outputs.float(), label.long()).item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss/len(val_dataloader):.10f}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        torch.save(checkpoint, 'model.pth')
    
    wandb.log({"Valid loss": val_loss})

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hokage321xxx (team-thay-linh). Use `wandb login --relogin` to force relogin


Epoch [1/50], Step [50/225], Train Loss: 1.1102343285
Epoch [1/50], Step [100/225], Train Loss: 0.6358348346
Epoch [1/50], Step [150/225], Train Loss: 0.4482467830
Epoch [1/50], Step [200/225], Train Loss: 0.3414552289
Epoch [1/50], Validation Loss: 0.2545958686
Epoch [2/50], Step [50/225], Train Loss: 0.3674542990
Epoch [2/50], Step [100/225], Train Loss: 0.1970529032
Epoch [2/50], Step [150/225], Train Loss: 0.1629444543
Epoch [2/50], Step [200/225], Train Loss: 0.1506556755
Epoch [2/50], Validation Loss: 0.1305352184
Epoch [3/50], Step [50/225], Train Loss: 0.1757016560
Epoch [3/50], Step [100/225], Train Loss: 0.1034808590
Epoch [3/50], Step [150/225], Train Loss: 0.0954039585
Epoch [3/50], Step [200/225], Train Loss: 0.0790156654
Epoch [3/50], Validation Loss: 0.0830792765
Epoch [4/50], Step [50/225], Train Loss: 0.1191307406
Epoch [4/50], Step [100/225], Train Loss: 0.0737433868
Epoch [4/50], Step [150/225], Train Loss: 0.0769483120
Epoch [4/50], Step [200/225], Train Loss: 0.069

KeyboardInterrupt: 